<a href="https://colab.research.google.com/github/jacksonStone/sentiment-analysis-with-hot-one-encoding-2/blob/main/DataGeneratorExperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq

First we need to generate a test data set.
I leverage Grok and the latest Llama 3 to do this.

In [ ]:
from groq import Groq
from google.colab import userdata
from time import sleep

client = Groq(
    api_key=userdata.get("GROQ_API_KEY"),
)
betterPositiveCompletions = []
for j in range(10):
  if(j > 0):
    sleep(50)
  for i in range(30):
    chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": "Generate 10 random positive statements. Be succient. One sentence per statement. Go straight to the content with no preamble. Structure them like so: STATEMENT:some positive statement.\nSTATEMENT:some other positive statement\netc.",
          }
      ],
      model="llama3-70b-8192",
    )
    betterPositiveCompletions.append(chat_completion.choices[0].message.content)
  print("finished batch: " + str(j) + " out of 100")
print(betterPositiveCompletions)
# persist chatCompletions to pickle
parsedPositiveCompletions = []
for i in betterPositiveCompletions:
  parsedPositiveCompletions += i.split("\nSTATEMENT:")[1:]
print(parsedPositiveCompletions)

betterNegativeCompletions = []
for j in range(10):
  if(j > 0):
    sleep(50)
  for i in range(30):
    chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": "Generate 10 random negative statements. Be succient. One sentence per statement. Go straight to the content with no preamble. Structure them like so: STATEMENT:some negative statement.\nSTATEMENT:some other negative statement\netc.",
          }
      ],
      model="llama3-70b-8192",
    )
    betterNegativeCompletions.append(chat_completion.choices[0].message.content)
  print("finished batch: " + str(j) + " out of 100")
print(betterNegativeCompletions)
# persist chatCompletions to pickle
parsedNegativeCompletions = []
for i in betterNegativeCompletions:
  parsedNegativeCompletions += i.split("\nSTATEMENT:")[1:]
print(parsedNegativeCompletions)

# persist chatCompletions to pickle
import pickle
with open('negativeChats2.pkl', 'wb') as f:
    pickle.dump(parsedNegativeCompletions, f)



finished batch: 0 out of 100
finished batch: 1 out of 100
finished batch: 2 out of 100
finished batch: 3 out of 100
finished batch: 4 out of 100
finished batch: 5 out of 100
finished batch: 6 out of 100
finished batch: 7 out of 100
finished batch: 8 out of 100
finished batch: 9 out of 100
["Here are 10 random positive statements:\n\nSTATEMENT:Every sunrise brings a new chance to start fresh and make today better than yesterday.\nSTATEMENT:You are stronger than you think, capable of achieving more than you imagine.\nSTATEMENT:Your unique talents and skills make you an essential puzzle piece in the grand tapestry of life.\nSTATEMENT:Every obstacle overcome is a testament to your unbreakable spirit and determination.\nSTATEMENT:You are doing the best you can, and that is something to be truly proud of.\nSTATEMENT:Your worth and value come from who you are as a person, not from what you do or achieve.\nSTATEMENT:Today is a brand new day full of possibilities and opportunities waiting to be

Then we save the data sets to Drive to prevent needing to rerun the 20min script

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/MyDrive/negativeChats.pkl', 'wb') as f:
    pickle.dump(parsedNegativeCompletions, f)

In [ ]:
with open('/content/drive/MyDrive/positiveChats.pkl', 'wb') as f:
    pickle.dump(parsedPositiveCompletions, f)

Now I need to convert the data into numerical values

In [ ]:
import string
import pickle
from random import shuffle
positiveChats = []
negativeChats = []
with open('/content/drive/MyDrive/positiveChats.pkl', 'rb') as f:
    positiveChats = dict.fromkeys(pickle.load(f)).keys()
with open('/content/drive/MyDrive/negativeChats.pkl', 'rb') as f:
    negativeChats = dict.fromkeys(pickle.load(f)).keys()
print("Positive chats: " + str(len(positiveChats)))
print("Negative chats: " + str(len(negativeChats)))
labeledPositiveChats = []
labeledNegativeChats = []
for chat in positiveChats:
  labeledPositiveChats.append((chat, "positive"))
for chat in negativeChats:
  labeledNegativeChats.append((chat, "negative"))
allChats = labeledPositiveChats + labeledNegativeChats
shuffle(allChats)
wordIndex = {}
counter = {"i": 0}
def convert_string_to_numbers(chat):
  numberifiedChat = []
  for word in chat.strip().split():
    word = word.lower()
    punctuation = ""
    if word[-1] in string.punctuation:
      punctuation = word[-1]
      if punctuation not in wordIndex:
        wordIndex[punctuation] = [counter["i"], 1]
        counter["i"] = counter["i"] + 1
      else:
        wordIndex[punctuation][1] += 1
      word = word[:-1]
    if len(word) > 0:
      if word not in wordIndex:
        wordIndex[word] = [counter["i"], 1]
        counter["i"] = counter["i"] + 1
      else:
        wordIndex[word][1] += 1
      numberifiedChat.append(wordIndex[word][0])
      if punctuation != "":
        numberifiedChat.append(wordIndex[punctuation][0])
  return numberifiedChat

print(len(allChats))
#filter out duplicative chats
chatsConvertedToNumbers = []
for (chat, sentiment) in allChats:
  numberifiedChat = convert_string_to_numbers(chat)
  if(len(numberifiedChat) < 21):
    chatsConvertedToNumbers.append((numberifiedChat, sentiment))
wordCount  = len(wordIndex.keys())
print("word count: " + str(wordCount))
print("Cleaned up test sentences: " + str(len(chatsConvertedToNumbers)))
print(allChats[:10])

Positive chats: 1792
Negative chats: 2286
4078
word count: 2397
Cleaned up test sentences: 4033
[('My health will never improve, no matter what I do.', 'negative'), ('This rain is going to ruin my entire week.', 'negative'), ('You are stronger than you think, and your resilience is a testament to your inner strength.\n', 'positive'), ('My favorite TV show just got cancelled after three seasons.', 'negative'), ('You are capable of achieving anything you set your mind to!', 'positive'), ('Believe in yourself and your abilities, and others will too.\n', 'positive'), ("Everyone I've ever trusted has ended up betraying me.", 'negative'), (' Her lack of confidence holds her back from achieving her goals.', 'negative'), ('Every new day brings a chance to learn, grow, and become a better version of yourself.\n', 'positive'), ("Life is full of beauty and wonder, and it's waiting to be explored.", 'positive')]


In [ ]:
# sort wordIndex by the second entry in each array in the 2d array
sortedWordIndex = sorted(wordIndex.items(), key=lambda x: x[1][1], reverse=True)
print(sortedWordIndex)
#substitute allChats first tuple entry with an array of numbers using the index value
for (chat, _) in allChats:
  for word in chat.strip().strip("\n").split():
    word = word.lower()


[('.', [10, 3987]), ('and', [25, 2256]), ('to', [16, 1670]), ('the', [101, 1578]), (',', [4, 1531]), ('is', [14, 1521]), ('a', [28, 1501]), ('you', [20, 1302]), ('your', [26, 891]), ('of', [42, 817]), ('are', [21, 784]), ('in', [49, 693]), ('every', [70, 614]), ('my', [0, 552]), ('new', [71, 434]), ('no', [6, 415]), ("i'm", [174, 370]), ('be', [86, 368]), ('life', [80, 363]), ('this', [12, 330]), ('make', [115, 317]), ('i', [9, 313]), ('day', [72, 313]), ('never', [3, 302]), ('has', [58, 284]), ('world', [204, 263]), ('for', [241, 262]), ('me', [62, 251]), ('will', [2, 243]), ('can', [142, 238]), ('with', [147, 231]), ('matter', [7, 215]), ('just', [35, 215]), ('start', [276, 203]), ('opportunity', [151, 198]), ('how', [183, 197]), ("i'll", [225, 190]), ('than', [23, 189]), ('not', [131, 188]), ('good', [254, 180]), ('from', [68, 175]), ('it', [207, 172]), ('enough', [346, 167]), ('always', [98, 164]), ('today', [197, 162]), ('an', [95, 157]), ('have', [118, 157]), ('fresh', [277, 156]

In [ ]:
# check if there are any repeats in allChats
for i in range(len(allChats)):
  for j in range(i+1, len(allChats)):
    if allChats[i][0] == allChats[j][0]:
      print(allChats[i])

I covert from From word-indexes to one-hot-vectors. The theory is each word has a unique meaning that is unknown to my model. This seemed better than the rather brittle embedding approaches I was seening in the literature.

In [ ]:
import numpy as np
chats_to_hot_one_formats = np.zeros(shape=(len(chatsConvertedToNumbers),20,wordCount+50), dtype="uint8")
count = 0
for i in chatsConvertedToNumbers:
  two_d_array = chats_to_hot_one_formats[count]
  count += 1
  word = 0
  for wordNumber in i[0]:
    two_d_array[word][wordNumber] = 1
    word += 1
print(chats_to_hot_one_formats)

def convert_string_to_hot_one(chat):
  hot_one_array = np.zeros(shape=(20,wordCount+50), dtype="uint8")
  word = 0

  numberified = convert_string_to_numbers(chat)
  for wordNumber in numberified:
    hot_one_array[word][wordNumber] = 1
    word += 1
  return hot_one_array

[[[1 0 0 ... 0 0 0]
  [0 1 0 ... 0 0 0]
  [0 0 1 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


Now for Defining the model.

In [ ]:
!pip install torch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the neural network model
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(nn.Linear((wordCount+50) * 20, 20))

        # Repeating dense layers
        for _ in range(1):
            self.layers.append(nn.Linear(20, 20))

        # Output layer
        self.output_layer = nn.Linear(20, 1)

    def forward(self, x):
        x = x.view(-1, (wordCount+50) * 20)  # Flatten the input
        for layer in self.layers:
            x = torch.relu(layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# Instantiate the model
model = BinaryClassificationModel()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Example function to train the model on one batch
def train_model(model, criterion, optimizer, test_data, test_labels, val_data, val_labels, epochs=20):
    for epoch in range(epochs):
        model.train()

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(test_data)
        loss = criterion(outputs, test_labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # check against validation set
        model.eval()
        with torch.no_grad():
            val_outputs = model(val_data)
            val_loss = criterion(val_outputs, val_labels)
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

Put labels in the correct format

In [ ]:
labelOfChats = []
for (chat, sentiment) in chatsConvertedToNumbers:
  if sentiment == "positive":
    labelOfChats.append([1])
  else:
    labelOfChats.append([0])
# Example labels (replace with your actual labels)
labels = torch.tensor(np.array(labelOfChats), dtype=torch.float32)  # Convert to a PyTorch tensor


Divide test and validation set, about 50/50 seemed good

In [ ]:

# Convert NumPy matrices to a PyTorch tensor
data_tensor = torch.tensor(chats_to_hot_one_formats, dtype=torch.float32)  # Convert to a PyTorch tensor
data_tensor = data_tensor.view(len(chats_to_hot_one_formats), -1)  # Reshape to (num_samples, 2500 * 20)

# split data_tensor to leave off the last 1000 elements
division = -1500
test_data = data_tensor[:division]
test_labels = labels[:division]
val_data = data_tensor[division:]
val_labels = labels[division:]
# Train the model
train_model(model, criterion, optimizer, test_data, test_labels, val_data, val_labels)

Epoch [1/20], Loss: 0.6882, Validation Loss: 0.6761
Epoch [2/20], Loss: 0.6738, Validation Loss: 0.6558
Epoch [3/20], Loss: 0.6504, Validation Loss: 0.6278
Epoch [4/20], Loss: 0.6185, Validation Loss: 0.5916
Epoch [5/20], Loss: 0.5775, Validation Loss: 0.5475
Epoch [6/20], Loss: 0.5273, Validation Loss: 0.4965
Epoch [7/20], Loss: 0.4689, Validation Loss: 0.4396
Epoch [8/20], Loss: 0.4036, Validation Loss: 0.3771
Epoch [9/20], Loss: 0.3332, Validation Loss: 0.3137
Epoch [10/20], Loss: 0.2635, Validation Loss: 0.2542
Epoch [11/20], Loss: 0.1999, Validation Loss: 0.2018
Epoch [12/20], Loss: 0.1458, Validation Loss: 0.1577
Epoch [13/20], Loss: 0.1025, Validation Loss: 0.1219
Epoch [14/20], Loss: 0.0695, Validation Loss: 0.0937
Epoch [15/20], Loss: 0.0456, Validation Loss: 0.0721
Epoch [16/20], Loss: 0.0290, Validation Loss: 0.0560
Epoch [17/20], Loss: 0.0180, Validation Loss: 0.0442
Epoch [18/20], Loss: 0.0110, Validation Loss: 0.0359
Epoch [19/20], Loss: 0.0066, Validation Loss: 0.0301
Ep

Now I can spot check values in the model


In [ ]:
# test the model against a new string
output = model(torch.tensor(convert_string_to_hot_one("You are nice."), dtype=torch.float32).view(1, -1))
output = output.detach().numpy()
if(output[0][0] > 0.5):
  print("positive")
else:
  print("negative")

positive
